# RAG
В этом ноутбуке реализован подход Retrieval-Augmented Generation (RAG) для генерации описаний одежды. Возможности генеративной модели комбинируются с релевантными примерами из обучающего датасета.  

Для этого используется:
- модель `SentenceTransformers` модель для построения эмбеддингов входных характеристик
- FAISS-индекс для поиска ближайших соседей
- извлеченные примеры, которые вместе с входными характеристиками подаются в промпт модели.  

Цель — повысить качество описаний за счёт ориентации на реальные примеры и понизить вероятность галлюцинаций.  

In [1]:
!pip install sentence-transformers faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 71.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 89.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 66.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 66.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 68.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

## 1. Загрузка и подготовка данных

In [2]:
import faiss
import pandas as pd
import numpy as np
import openai
from sentence_transformers import SentenceTransformer

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# беру те же датасеты, что и в rut5, gpt
df_train = pd.read_csv('drive/MyDrive/proj_cloth_desc_gen/df_train.csv', index_col =0).reset_index(drop=True)
df_val = pd.read_csv('drive/MyDrive/proj_cloth_desc_gen/df_val.csv', index_col =0).reset_index(drop=True)

In [5]:
# удаляю инструкцию
df_train["input"] = df_train["input"].str.replace("Сгенерируй описание одежды для карточки товара:\n", "", regex=False)
df_val["input"] = df_val["input"].str.replace("Сгенерируй описание одежды для карточки товара:\n", "", regex=False)

## 2. Подбор параметров RAG (SentenceTransformer + нормализация)
Сравним на нескольких примеров качество генерации и найденные похожие примеры для разных моделей построения эмбеддингов:
- all-MiniLM-L6-v2
- paraphrase-multilingual-MiniLM-L12-v2
  
А также с использованием и без использования нормализации эмбеддингов

In [6]:
# функция построения FAISS индекса
def build_faiss_index(model_name="all-MiniLM-L6-v2", normalize=False):

    # Переводим входные тексты в эмбеддинги
    model = SentenceTransformer(model_name)
    texts = df_train["input"].tolist()
    embeddings = model.encode(texts, convert_to_numpy=True)

    if normalize:
        # нормализуем эмбеддинги по длине (норма 1)
        embeddings = embeddings / np.linalg.norm(embeddings, axis=1, keepdims=True)
        # поиск ближайших соседей по косинусному сходству
        index = faiss.IndexFlatIP(embeddings.shape[1])
    else:
        # поиск по евклидовому расстоянию
        index = faiss.IndexFlatL2(embeddings.shape[1])

    # добавляем эмбеддинг в индекс
    index.add(embeddings)
    return index, embeddings, model


In [7]:
# функция поиска k ближайших примеров из train датасета на основе FAISS-индекса
def retrieve_similar_examples(query, index, model, normalize=False, k=3):
    
    query_emb = model.encode(query, convert_to_numpy=True).reshape(1, -1)
    if normalize:
        query_emb = query_emb / np.linalg.norm(query_emb, axis=1, keepdims=True)

    distances, indices = index.search(query_emb, k)
    return df_train.iloc[indices[0]]

# функция генерации описания при помощи RAG
def generate_description_rag(input_text, index, model, normalize=False, k=3):
    # K близжайших примеров на train датасете по входному тексту
    retrieved = retrieve_similar_examples(input_text, index, model, normalize, k)

    # добавляем инструкцию + похожие примеры + входной текст
    prompt = "Сгенерируй описание одежды по характеристикам. Не выдумывай того, чего нет.\n"
    for _, row in retrieved.iterrows():
        prompt += f"\nХарактеристики:\n{row['input']}\nОписание:\n{row['final_desc']}\n"

    prompt += f"\nХарактеристики:\n{input_text}\nОписание:\n"

    # генерация через OpenAI
    response = client.chat.completions.create(
        model="gpt-4",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.7,
        top_p=0.95,
        max_tokens=512,
        timeout=60
    )
    return retrieved, response.choices[0].message.content


In [8]:
from getpass import getpass
import openai

openai.api_key = getpass("Введи OpenAI API ключ: ")

Введи OpenAI API ключ: ··········


In [9]:
client = openai.OpenAI(api_key=openai.api_key)

In [10]:
# рандломные примеры
df_sample = df_val.sample(5, random_state = 42)
inp = df_sample['input'].tolist()

In [11]:
# all-MiniLM-L6-v2 без нормализации
idx_1, emb_1, model_1 = build_faiss_index(model_name="all-MiniLM-L6-v2", normalize=False)
emb_1.shape

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

(44727, 384)

In [29]:
serch_list, gen_list = [], []
for i in inp:
  df_serch, gen_rag = generate_description_rag(i, idx_1, model_1, normalize=False, k=3)
  serch_list.append(df_serch['input'].tolist())
  gen_list.append(gen_rag)
df_sample['l6_k'] = serch_list
df_sample['l6_gen'] = gen_list

In [13]:
# all-MiniLM-L6-v2 с нормализацией
idx_2, emb_2, model_2 = build_faiss_index(model_name="all-MiniLM-L6-v2", normalize=True)
emb_2.shape

(44727, 384)

In [28]:
serch_list, gen_list = [], []
for i in inp:
  df_serch, gen_rag = generate_description_rag(i, idx_2, model_2, normalize=True, k=3)
  serch_list.append(df_serch['input'].tolist())
  gen_list.append(gen_rag)
df_sample['l6_norm_k'] = serch_list
df_sample['l6_norm_gen'] = gen_list

In [30]:
# paraphrase-multilingual-MiniLM-L12-v2 без нормализации
idx_3, emb_3, model_3 = build_faiss_index(model_name="paraphrase-multilingual-MiniLM-L12-v2", normalize=False)
emb_3.shape

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/645 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

(44727, 384)

In [31]:
serch_list, gen_list = [], []
for i in inp:
  df_serch, gen_rag = generate_description_rag(i, idx_3, model_3, normalize=False, k=3)
  serch_list.append(df_serch['input'].tolist())
  gen_list.append(gen_rag)
df_sample['l12_k'] = serch_list
df_sample['l12_gen'] = gen_list

In [32]:
# paraphrase-multilingual-MiniLM-L12-v2 с нормализацией
idx_4, emb_4, model_4 = build_faiss_index(model_name="paraphrase-multilingual-MiniLM-L12-v2", normalize=True)
emb_4.shape

(44727, 384)

In [33]:
serch_list, gen_list = [], []
for i in inp:
  df_serch, gen_rag = generate_description_rag(i, idx_4, model_4, normalize=True, k=3)
  serch_list.append(df_serch['input'].tolist())
  gen_list.append(gen_rag)
df_sample['l12_norm_k'] = serch_list
df_sample['l12_norm_gen'] = gen_list

In [37]:
for _, row in df_sample.iterrows():
  print(f"Input text:\n{row['input']}\n")
  print(f"True target:\n{row['final_desc']}\n")
  print(f"all-MiniLM-L6-v2:\n{row['l6_gen']}\n")
  print(f"all-MiniLM-L6-v2 normalized:\n{row['l6_norm_gen']}\n")
  print(f"paraphrase-multilingual-MiniLM-L12-v2:\n{row['l12_gen']}\n")
  print(f"paraphrase-multilingual-MiniLM-L12-v2 normalized:\n{row['l12_norm_gen']}\n")

Input text:
Наименование товара: Брюки классические прямые
Сезон: круглогодичный
Особенности модели: дышащий материал; немнущийся материал; эластичный материал
Тип посадки: средняя
Страна производства: Турция
Вид застежки: молния
Тип карманов: прорезные; косые
Комплектация: брюки - 1 шт.
Декоративные элементы: без стрелок

True target:
Брюки мужские классические демисезонные. Осенние костюмные брюки поразят вас своеи универсальностью, ведь они почти не мнутся и требуют минимального дополнительного ежедневного ухода. Такие повседневные штаны подходят для парнеи и мужчин, позволят быть одновременно стильными и брутальными. Прямые брюки без стрелок идеальны как праздничные для похода на свадьбу или юбилеи и как деловые для похода в офис на работу. Черные брюки будут оценены школьниками и студентами. Даже если раньше вы не носили классику в школу или универ, то сеичас не сможете устоять. Наши всесезонные брюки немного тянутся, имеют передние косые и задние прорезные карманы. Вечерние брюки

В целом качество генерации для всех конфигураций довольно приличное. Серьезные галлюцинации практически отсутсвуют.
Иногда проскакивают мелкие галлюцинации — например:
- выдумывает состав: «100% хлопок», «натуральный шелк»
- приписывает качества: «премиум-материалы», «долговечность», «сохраняет форму»

Нормализованные модели дают более шаблонный и менее детализированный текст  
Ненормализованные модели дают больше конкретики, тексты ближе по стилю к таргету 


Теперь посмотрим на релевантность близжайших примеров

In [38]:
for _, row in df_sample.iterrows():
  print(f"Input text:\n{row['input']}\n")
  print(f"all-MiniLM-L6-v2:\n{row['l6_k']}\n")
  print(f"all-MiniLM-L6-v2 normalized:\n{row['l6_norm_k']}\n")
  print(f"paraphrase-multilingual-MiniLM-L12-v2:\n{row['l12_k']}\n")
  print(f"paraphrase-multilingual-MiniLM-L12-v2 normalized:\n{row['l12_norm_k']}\n")

Input text:
Наименование товара: Брюки классические прямые
Сезон: круглогодичный
Особенности модели: дышащий материал; немнущийся материал; эластичный материал
Тип посадки: средняя
Страна производства: Турция
Вид застежки: молния
Тип карманов: прорезные; косые
Комплектация: брюки - 1 шт.
Декоративные элементы: без стрелок

all-MiniLM-L6-v2:
['Наименование товара: Брюки классические зауженные\nСезон: круглогодичный\nОсобенности модели: дышащий материал\nТип посадки: средняя\nСтрана производства: Китай\nВид застежки: пуговицы; молния\nТип карманов: прорезные\nКомплектация: брюки мужские - 1 шт\nДекоративные элементы: без элементов', 'Наименование товара: Брюки мужские утепленные оверсайз спортивные\nСезон: круглогодичный\nОсобенности модели: дышащий материал; утепленные; с начесом\nТип посадки: средняя\nСтрана производства: Узбекистан\nВид застежки: завязки; шнуровка\nТип карманов: прорезные\nКомплектация: Брюки - 1 шт.\nДекоративные элементы: без элементов; молния; вышивка', 'Наименован

Видно, что `paraphrase-multilingual-MiniLM-L12-v2` дает более релевантные примеры, чем `all-MiniLM-L6-v2`

Например, для «Праздничное платье рубашка на Новый год приталенное» `all-MiniLM-L6-v2` выдает в качестве близжайших примеров:

1. Приталенная рубашка корсетного типа
2. Туника вязаная больших размеров
3. Пальто классическое демисезонное
   
что не совсем релевантно для рассматриваемого примера

Так как качество подобранных примеров определяет стабильность RAG, выбираю paraphrase-multilingual-MiniLM-L12-v2.  
Ненормализованная версия даёт более богатый и рекламный стиль описаний при сохранении приемлемой фактичности, поэтому выбираю paraphrase-multilingual-MiniLM-L12-v2 без нормализации.

In [39]:
# сохраняем индекс
faiss.write_index(idx_3, "drive/MyDrive/proj_cloth_desc_gen/final_idx.index")

In [57]:
# сохраняем эмбеддинги
df_embeddings = pd.DataFrame({
    "text": df_train["input"].tolist(),
    "embedding": [emb.tolist() for emb in emb_3],
    "final_desc": df_train["final_desc"].tolist(),
})
df_embeddings.to_parquet("drive/MyDrive/proj_cloth_desc_gen/embeddings.parquet")

## 3. Генерация текста
Беру сэмпл из 200 примеров из тестовой выборки. Для каждого примера генерирую описание и сохраняю результат в CSV. 

In [48]:
index = faiss.read_index("drive/MyDrive/proj_cloth_desc_gen/final_idx.index")

In [52]:
df_test =  pd.read_csv('drive/MyDrive/proj_cloth_desc_gen/df_sample_pred.csv', index_col =0).reset_index(drop=True)
df_test["input"] = df_test["input"].str.replace("Сгенерируй описание одежды для карточки товара:\n", "", regex=False)

In [58]:
df_embeddings = pd.read_parquet("drive/MyDrive/proj_cloth_desc_gen/embeddings.parquet")
df_embeddings

,text,embedding,final_desc
0,Наименование товара: Костюм Деда Мороза\nОсобе...,"[-0.21763770282268524, 0.250986784696579, 0.06...",Предлагаем вашему вниманию идеальныи мужскои в...
1,Наименование товара: Куртка зимняя с капюшоном...,"[-0.16727910935878754, 0.12410808354616165, 0....",DESPORT это качественные и доступные спортивны...
2,Наименование товара: Однотонная футболка поло ...,"[0.22782951593399048, 0.3239396810531616, -0.0...",Рубашка поло с коротким рукавом прямого кроя с...
3,Наименование товара: Зимняя куртка (бушлат) ВК...,"[-0.18706287443637848, 0.22443805634975433, 0....",Куртка зимняя ВКБОВКПО предназначена для испол...
4,Наименование товара: Новогодняя футболка с над...,"[-0.15429887175559998, 0.26661792397499084, -0...",Женская футболка от бренда со стразами это иде...
...,...,...,...
44722,Наименование товара: Топ бандо без лямок\nПокр...,"[-0.10823718458414078, 0.1371346116065979, 0.0...",Топ женскии бандо без чашек от бренда - это кл...
44723,Наименование товара: Толстовка на молнии оверс...,"[-0.15855193138122559, 0.030281798914074898, 0...",Толстовка женская на молнии с принтом диснеи м...
44724,Наименование товара: Прозрачный лонгслив облег...,"[-0.08728830516338348, 0.22507517039775848, 0....",Полупрозрачныи лонгслив это отличныи выбор для...
44725,Наименование товара: Свитер свободный с грибам...,"[-0.12986910343170166, 0.1911398321390152, 0.0...","Свитер изготовлен в России, универсальныи свит..."


In [60]:
model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")

In [59]:
# функция поиска k ближайших примеров 
def retrieve_similar_examples(query, index, model, df, normalize=False, k=3):
    query_emb = model.encode(query, convert_to_numpy=True).reshape(1, -1)
    if normalize:
        query_emb = query_emb / np.linalg.norm(query_emb, axis=1, keepdims=True)

    distances, indices = index.search(query_emb, k)
    return df.iloc[indices[0]]

# функция генерации описания при помощи RAG
def generate_description_rag(input_text, index, model, df, normalize=False, k=3):
    retrieved = retrieve_similar_examples(input_text, index, model, df, normalize, k)

    prompt = "Сгенерируй описание одежды по характеристикам. Не выдумывай того, чего нет.\n"
    for _, row in retrieved.iterrows():
        prompt += f"\nХарактеристики:\n{row['text']}\nОписание:\n{row['final_desc']}\n"

    prompt += f"\nХарактеристики:\n{input_text}\nОписание:\n"

    response = client.chat.completions.create(
        model="gpt-4",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.7,
        top_p=0.95,
        max_tokens=512,
        timeout=60
    )
    return retrieved, response.choices[0].message.content


In [61]:
# генерирую описания на тесте
from tqdm import tqdm

gen_list = []
for i, row in tqdm(df_test.iterrows(), total=len(df_test)):
    try:
      __, gen_rag = generate_description_rag(row["input"], index, model, df_embeddings, normalize=False, k=3)
      gen_list.append(gen_rag)

    except Exception as e:
        print(f"Ошибка на {i}: {e}")
        gen_list.append("")


 80%|███████▉  | 159/200 [49:04<07:30, 10.98s/it]

Ошибка на 158: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}


 80%|████████  | 160/200 [49:06<05:27,  8.19s/it]

Ошибка на 159: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}


 80%|████████  | 161/200 [49:08<04:03,  6.24s/it]

...

Ошибка на 197: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}


100%|█████████▉| 199/200 [50:09<00:01,  1.79s/it]

Ошибка на 198: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}


100%|██████████| 200/200 [50:10<00:00, 15.05s/it]

Ошибка на 199: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}


In [62]:
df_test['rag_gen'] = gen_list
df_test.to_csv('drive/MyDrive/proj_cloth_desc_gen/df_sample_pred.csv')